In [1]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv3D, MaxPooling3D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

import os
from keras import backend as K

from sklearn.model_selection import train_test_split
import numpy as np

landmarks = [
    'PCLOrigin',
    'lateralCondyle',
    'medialCondyle',
    'medialSulcus',
    'whitesideReference'
]

Using TensorFlow backend.


In [2]:
def one_axis_femur_model(name='PCLOrigin', axis=0, data_size = 15):
    output_path = "/code/" + name + "/"
    imgs_to_process = []
    lbls_to_process = []
    dif_arr = [(0,1,2), (0,2,1), (1,0,2), (1,2,0), (2,0,1), (2,1,0)]
    num_classes = 120

    for index in range(data_size):
        for a, b, c in dif_arr:
            imgs_to_process.append(np.load(output_path + 'images/' + '{}-{}-{}-{}.npy'.format(index, a, b, c)))
            label = np.load(output_path + 'label/' + '{}-{}-{}-{}.npy'.format(index, a, b, c))
            lbls_to_process.append(int(label[3+axis]))
            num_classes = int(label[6]) * 2

    imgs_to_process = np.array(imgs_to_process)
    lbls_to_process = np.array(lbls_to_process)

    batch_size = 15
    epochs = 200
    learning_rate = 0.001

    x_train, x_test, y_train, y_test = train_test_split(imgs_to_process, lbls_to_process, test_size=0.2)
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    img_rows = x_train.shape[1]
    img_cols = x_train.shape[2]
    colors = x_train.shape[3]


    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1,colors, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 1,colors, img_rows, img_cols)
        input_shape = (1, colors, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, colors, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, colors, 1)
        input_shape = (img_rows, img_cols, colors, 1)

    print(num_classes)
    # Convert class vectors to binary class matrices.
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)


    model = Sequential()
    model.add(Conv3D(3, kernel_size=(3, 3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv3D(3, kernel_size=(3, 3, 3), activation='relu'))
    model.add(Conv3D(3, kernel_size=(3, 3, 3), activation='relu'))
    model.add(Conv3D(3, kernel_size=(3, 3, 3), activation='relu'))
    model.add(Conv3D(3, kernel_size=(3, 3, 3), activation='relu'))
    model.add(Conv3D(3, kernel_size=(3, 3, 3), activation='relu'))
    model.add(Conv3D(3, kernel_size=(1, 1, 1)))




    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(3,3,3)))

    model.add(Flatten())
    # Fully connected layer

    BatchNormalization()
    model.add(Dense(512))
    model.add(Activation('relu'))
    BatchNormalization()
    model.add(Dropout(0.2))
    model.add(Dense(num_classes))

    model.add(Activation('softmax'))


    sgd=SGD(lr=learning_rate)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=sgd,
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))

    score = model.evaluate(x_test, y_test, verbose=0)
    model.save('one_axis_femur_model_{}_{}.h5'.format(name, axis))
    prediction = model.predict(x_test)
    print(prediction)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    print(score)
    print(y_test)


In [3]:
one_axis_femur_model('medialSulcus', 0, 40)

x_train shape: (192, 42, 42, 42)
192 train samples
48 test samples
192 train samples
48 test samples
42
Train on 192 samples, validate on 48 samples
Epoch 1/200
192/192 [==============================] - 16s 83ms/step - loss: 4.3506 - accuracy: 0.0833 - val_loss: 2.7980 - val_accuracy: 0.0833
Epoch 2/200
192/192 [==============================] - 16s 81ms/step - loss: 2.8331 - accuracy: 0.2240 - val_loss: 2.6400 - val_accuracy: 0.1250
Epoch 3/200
192/192 [==============================] - 16s 81ms/step - loss: 2.4788 - accuracy: 0.2188 - val_loss: 2.4604 - val_accuracy: 0.0833
Epoch 4/200
192/192 [==============================] - 16s 82ms/step - loss: 2.3343 - accuracy: 0.2396 - val_loss: 2.8587 - val_accuracy: 0.1250
Epoch 5/200
192/192 [==============================] - 16s 82ms/step - loss: 2.3476 - accuracy: 0.2292 - val_loss: 2.2928 - val_accuracy: 0.1667
Epoch 6/200
192/192 [==============================] - 16s 82ms/step - loss: 2.1832 - accuracy: 0.2604 - val_loss: 2.3363 - va

192/192 [==============================] - 16s 81ms/step - loss: 0.0726 - accuracy: 0.9948 - val_loss: 3.0072 - val_accuracy: 0.3542
Epoch 57/200
192/192 [==============================] - 16s 82ms/step - loss: 0.0995 - accuracy: 0.9792 - val_loss: 3.0422 - val_accuracy: 0.3542
Epoch 58/200
192/192 [==============================] - 16s 82ms/step - loss: 0.0611 - accuracy: 0.9896 - val_loss: 3.0516 - val_accuracy: 0.3750
Epoch 59/200
192/192 [==============================] - 16s 82ms/step - loss: 0.0608 - accuracy: 1.0000 - val_loss: 3.1324 - val_accuracy: 0.3750
Epoch 60/200
192/192 [==============================] - 16s 81ms/step - loss: 0.0926 - accuracy: 0.9740 - val_loss: 3.2916 - val_accuracy: 0.3542
Epoch 61/200
192/192 [==============================] - 16s 81ms/step - loss: 0.0505 - accuracy: 1.0000 - val_loss: 3.2338 - val_accuracy: 0.3542
Epoch 62/200
192/192 [==============================] - 16s 82ms/step - loss: 0.0614 - accuracy: 0.9948 - val_loss: 3.3323 - val_accuracy

192/192 [==============================] - 16s 81ms/step - loss: 0.1183 - accuracy: 0.9740 - val_loss: 3.8404 - val_accuracy: 0.3542
Epoch 113/200
192/192 [==============================] - 16s 82ms/step - loss: 0.0484 - accuracy: 0.9844 - val_loss: 3.9797 - val_accuracy: 0.3333
Epoch 114/200
192/192 [==============================] - 16s 82ms/step - loss: 0.0153 - accuracy: 1.0000 - val_loss: 3.9288 - val_accuracy: 0.3958
Epoch 115/200
192/192 [==============================] - 16s 82ms/step - loss: 0.0118 - accuracy: 1.0000 - val_loss: 4.0059 - val_accuracy: 0.3958
Epoch 116/200
192/192 [==============================] - 16s 82ms/step - loss: 0.0124 - accuracy: 1.0000 - val_loss: 4.0687 - val_accuracy: 0.3958
Epoch 117/200
192/192 [==============================] - 16s 81ms/step - loss: 0.0097 - accuracy: 1.0000 - val_loss: 4.0757 - val_accuracy: 0.3958
Epoch 118/200
192/192 [==============================] - 16s 82ms/step - loss: 0.0220 - accuracy: 0.9948 - val_loss: 4.7415 - val_ac

Epoch 168/200
192/192 [==============================] - 16s 81ms/step - loss: 0.0105 - accuracy: 0.9948 - val_loss: 4.6296 - val_accuracy: 0.3125
Epoch 169/200
192/192 [==============================] - 16s 82ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 4.5439 - val_accuracy: 0.3125
Epoch 170/200
192/192 [==============================] - 16s 81ms/step - loss: 0.0061 - accuracy: 1.0000 - val_loss: 4.5695 - val_accuracy: 0.2917
Epoch 171/200
192/192 [==============================] - 16s 81ms/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 4.5766 - val_accuracy: 0.2917
Epoch 172/200
192/192 [==============================] - 16s 81ms/step - loss: 0.0042 - accuracy: 1.0000 - val_loss: 4.6395 - val_accuracy: 0.2917
Epoch 173/200
192/192 [==============================] - 16s 82ms/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 4.6426 - val_accuracy: 0.2917
Epoch 174/200
192/192 [==============================] - 16s 81ms/step - loss: 0.0073 - accuracy: 1.0000 - val_loss: 4

In [4]:
one_axis_femur_model('medialSulcus', 1, 40)

x_train shape: (192, 42, 42, 42)
192 train samples
48 test samples
192 train samples
48 test samples
42
Train on 192 samples, validate on 48 samples
Epoch 1/200
192/192 [==============================] - 16s 82ms/step - loss: 4.2359 - accuracy: 0.0833 - val_loss: 3.7920 - val_accuracy: 0.0833
Epoch 2/200
192/192 [==============================] - 16s 82ms/step - loss: 3.5834 - accuracy: 0.1510 - val_loss: 3.6104 - val_accuracy: 0.1875
Epoch 3/200
192/192 [==============================] - 16s 82ms/step - loss: 3.2907 - accuracy: 0.2188 - val_loss: 3.2038 - val_accuracy: 0.1875
Epoch 4/200
192/192 [==============================] - 16s 82ms/step - loss: 2.8698 - accuracy: 0.1823 - val_loss: 2.7165 - val_accuracy: 0.1458
Epoch 5/200
192/192 [==============================] - 16s 82ms/step - loss: 2.3453 - accuracy: 0.2135 - val_loss: 2.4818 - val_accuracy: 0.1875
Epoch 6/200
192/192 [==============================] - 16s 81ms/step - loss: 2.2665 - accuracy: 0.1771 - val_loss: 2.4176 - va

192/192 [==============================] - 16s 82ms/step - loss: 1.0615 - accuracy: 0.5781 - val_loss: 3.5175 - val_accuracy: 0.1667
Epoch 57/200
192/192 [==============================] - 16s 81ms/step - loss: 1.0704 - accuracy: 0.5990 - val_loss: 3.5498 - val_accuracy: 0.2083
Epoch 58/200
192/192 [==============================] - 16s 81ms/step - loss: 1.0716 - accuracy: 0.6354 - val_loss: 3.4839 - val_accuracy: 0.1875
Epoch 59/200
192/192 [==============================] - 16s 82ms/step - loss: 1.1047 - accuracy: 0.5938 - val_loss: 3.3441 - val_accuracy: 0.2292
Epoch 60/200
192/192 [==============================] - 16s 82ms/step - loss: 0.9693 - accuracy: 0.6667 - val_loss: 3.7015 - val_accuracy: 0.2083
Epoch 61/200
192/192 [==============================] - 16s 82ms/step - loss: 1.1450 - accuracy: 0.6198 - val_loss: 3.4417 - val_accuracy: 0.1875
Epoch 62/200
192/192 [==============================] - 16s 81ms/step - loss: 0.9737 - accuracy: 0.6302 - val_loss: 3.7348 - val_accuracy

192/192 [==============================] - 16s 81ms/step - loss: 0.5150 - accuracy: 0.8073 - val_loss: 7.7489 - val_accuracy: 0.2083
Epoch 113/200
192/192 [==============================] - 16s 81ms/step - loss: 0.6364 - accuracy: 0.8021 - val_loss: 5.8232 - val_accuracy: 0.2292
Epoch 114/200
192/192 [==============================] - 16s 81ms/step - loss: 0.6808 - accuracy: 0.7448 - val_loss: 4.4340 - val_accuracy: 0.1875
Epoch 115/200
192/192 [==============================] - 16s 81ms/step - loss: 0.6605 - accuracy: 0.7552 - val_loss: 5.8354 - val_accuracy: 0.1875
Epoch 116/200
192/192 [==============================] - 16s 81ms/step - loss: 0.4839 - accuracy: 0.8438 - val_loss: 6.5730 - val_accuracy: 0.1875
Epoch 117/200
192/192 [==============================] - 16s 81ms/step - loss: 0.5602 - accuracy: 0.8021 - val_loss: 6.3944 - val_accuracy: 0.1667
Epoch 118/200
192/192 [==============================] - 16s 82ms/step - loss: 0.5373 - accuracy: 0.8073 - val_loss: 6.5188 - val_ac

Epoch 174/200
192/192 [==============================] - 16s 82ms/step - loss: 0.4828 - accuracy: 0.8854 - val_loss: 11.0499 - val_accuracy: 0.2083
Epoch 175/200
192/192 [==============================] - 16s 81ms/step - loss: 0.2844 - accuracy: 0.8958 - val_loss: 12.0770 - val_accuracy: 0.1875
Epoch 176/200
192/192 [==============================] - 16s 82ms/step - loss: 0.2841 - accuracy: 0.8958 - val_loss: 10.9598 - val_accuracy: 0.2083
Epoch 177/200
192/192 [==============================] - 16s 81ms/step - loss: 0.2340 - accuracy: 0.9062 - val_loss: 14.2021 - val_accuracy: 0.2083
Epoch 178/200
192/192 [==============================] - 16s 81ms/step - loss: 0.1922 - accuracy: 0.9323 - val_loss: 15.0336 - val_accuracy: 0.1667
Epoch 179/200
192/192 [==============================] - 16s 81ms/step - loss: 0.1882 - accuracy: 0.9427 - val_loss: 15.1001 - val_accuracy: 0.1458
Epoch 180/200
192/192 [==============================] - 16s 82ms/step - loss: 0.1982 - accuracy: 0.9219 - val_l

In [5]:
one_axis_femur_model('medialSulcus', 2, 40)

x_train shape: (192, 42, 42, 42)
192 train samples
48 test samples
192 train samples
48 test samples
42
Train on 192 samples, validate on 48 samples
Epoch 1/200
192/192 [==============================] - 16s 82ms/step - loss: 4.4244 - accuracy: 0.0417 - val_loss: 3.7438 - val_accuracy: 0.0833
Epoch 2/200
192/192 [==============================] - 16s 82ms/step - loss: 3.6604 - accuracy: 0.0365 - val_loss: 3.6601 - val_accuracy: 0.1250
Epoch 3/200
192/192 [==============================] - 16s 81ms/step - loss: 3.4750 - accuracy: 0.1667 - val_loss: 3.3392 - val_accuracy: 0.2083
Epoch 4/200
192/192 [==============================] - 16s 81ms/step - loss: 2.9991 - accuracy: 0.2135 - val_loss: 2.7164 - val_accuracy: 0.2292
Epoch 5/200
192/192 [==============================] - 16s 81ms/step - loss: 2.4618 - accuracy: 0.2500 - val_loss: 2.3825 - val_accuracy: 0.2500
Epoch 6/200
192/192 [==============================] - 16s 82ms/step - loss: 2.2093 - accuracy: 0.2448 - val_loss: 2.3503 - va

192/192 [==============================] - 16s 81ms/step - loss: 1.2920 - accuracy: 0.5104 - val_loss: 2.7876 - val_accuracy: 0.2083
Epoch 57/200
192/192 [==============================] - 16s 82ms/step - loss: 1.2696 - accuracy: 0.5156 - val_loss: 2.9688 - val_accuracy: 0.2083
Epoch 58/200
192/192 [==============================] - 16s 82ms/step - loss: 1.3830 - accuracy: 0.4844 - val_loss: 2.5894 - val_accuracy: 0.2500
Epoch 59/200
192/192 [==============================] - 16s 82ms/step - loss: 1.2659 - accuracy: 0.5260 - val_loss: 2.7118 - val_accuracy: 0.2500
Epoch 60/200
192/192 [==============================] - 16s 81ms/step - loss: 1.1791 - accuracy: 0.5729 - val_loss: 2.7583 - val_accuracy: 0.2500
Epoch 61/200
192/192 [==============================] - 16s 82ms/step - loss: 1.1898 - accuracy: 0.5365 - val_loss: 2.8357 - val_accuracy: 0.1667
Epoch 62/200
192/192 [==============================] - 16s 81ms/step - loss: 1.2447 - accuracy: 0.5729 - val_loss: 2.7707 - val_accuracy

192/192 [==============================] - 16s 82ms/step - loss: 0.9847 - accuracy: 0.6094 - val_loss: 4.1812 - val_accuracy: 0.2292
Epoch 113/200
192/192 [==============================] - 17s 89ms/step - loss: 0.9549 - accuracy: 0.6302 - val_loss: 4.2061 - val_accuracy: 0.2917
Epoch 114/200
192/192 [==============================] - 18s 94ms/step - loss: 0.9285 - accuracy: 0.6562 - val_loss: 4.2579 - val_accuracy: 0.2500
Epoch 115/200
192/192 [==============================] - 16s 81ms/step - loss: 1.1752 - accuracy: 0.5938 - val_loss: 4.2016 - val_accuracy: 0.1875
Epoch 116/200
192/192 [==============================] - 16s 81ms/step - loss: 0.9910 - accuracy: 0.6250 - val_loss: 4.3961 - val_accuracy: 0.2708
Epoch 117/200
192/192 [==============================] - 16s 81ms/step - loss: 0.9729 - accuracy: 0.6302 - val_loss: 4.5294 - val_accuracy: 0.2708
Epoch 118/200
192/192 [==============================] - 16s 81ms/step - loss: 1.0355 - accuracy: 0.6198 - val_loss: 4.7726 - val_ac

Epoch 168/200
192/192 [==============================] - 16s 81ms/step - loss: 0.6316 - accuracy: 0.7604 - val_loss: 7.8868 - val_accuracy: 0.2083
Epoch 169/200
192/192 [==============================] - 16s 82ms/step - loss: 0.5434 - accuracy: 0.7917 - val_loss: 7.7978 - val_accuracy: 0.2292
Epoch 170/200
192/192 [==============================] - 16s 82ms/step - loss: 0.5671 - accuracy: 0.7812 - val_loss: 8.0827 - val_accuracy: 0.2292
Epoch 171/200
192/192 [==============================] - 16s 81ms/step - loss: 0.5338 - accuracy: 0.7969 - val_loss: 8.3081 - val_accuracy: 0.2708
Epoch 172/200
192/192 [==============================] - 16s 81ms/step - loss: 0.5559 - accuracy: 0.7812 - val_loss: 8.5245 - val_accuracy: 0.2292
Epoch 173/200
192/192 [==============================] - 16s 81ms/step - loss: 0.5448 - accuracy: 0.7969 - val_loss: 8.6698 - val_accuracy: 0.2292
Epoch 174/200
192/192 [==============================] - 16s 81ms/step - loss: 0.5504 - accuracy: 0.7969 - val_loss: 8